### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "German"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish German"

# parameter
file = "Turkish German SV"
sheets = "2 gram hybrid"  # 2 gram target, 2 gram hybrid
start_first_column = 0  # start column location number
time_shift = 0.6
sample_num = 6

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [7]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [8]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

KeyboardInterrupt: 

In [ ]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,word,twogram_pair1
0,seu,ama doktor
1,seu,evet madam
2,seu,çikolata mı
3,seu,bana şeker
4,seu,sinyal yok
5,seu,alkol yok
6,seu,onu tart
7,seu,beni duy
8,seu,bunu duy


In [ ]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[start_first_column:]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [ ]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,ama doktor,7038.167,7043.167,ama doktor dinlenmen gerektiğini söyledi bunda...,1yYN4FZDEGQ
1,ama doktor,4715.021,4717.583,ama doktor bahsetmişti zaten,N5rqYAWRdu4
2,ama doktor,2838.854,2843.854,ama doktor bey bize bol fındıkla kiraz suyu ön...,M6M7psZYUB4
3,ama doktor,466.000,471.610,ameliyatı kaç saat sürüyor diyorlar benimki be...,jYxyg1TrJZg
4,ama doktor,6140.285,6143.480,bir sevdiğin ama doktor vallaha aynen böyle dedi,z0cVY9navGM
5,ama doktor,6012.101,6013.780,ama doktor bir süre olur dedi,aSozR1GKefY
6,çikolata mı,449.450,483.320,evet şimdi kenar için hazırlamış olduğum çikol...,J8K0vB6HgrY
7,çikolata mı,87.774,93.732,lokum mu çikolata mı deseler,aI0JSNFZ5bA
8,çikolata mı,378.260,387.320,üzerlerindeki çikolata mı aldım bakın çok güze...,LfHpjk6K2hw
9,çikolata mı,419.045,424.445,telefonda şeker mi alsam çikolata mı alsam bak...,0AOYUEuSiyw


In [ ]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,ama doktor,7038.167000,7038.872128,ama doktor dinlenmen gerektiğini söyledi bunda...,1yYN4FZDEGQ
1,ama doktor,4715.021000,4716.027500,ama doktor bahsetmişti zaten,N5rqYAWRdu4
2,ama doktor,2838.854000,2839.770667,ama doktor bey bize bol fındıkla kiraz suyu ön...,M6M7psZYUB4
3,ama doktor,469.377449,470.064388,ameliyatı kaç saat sürüyor diyorlar benimki be...,jYxyg1TrJZg
4,ama doktor,6141.083750,6141.882500,bir sevdiğin ama doktor vallaha aynen böyle dedi,z0cVY9navGM
5,ama doktor,6012.101000,6012.737862,ama doktor bir süre olur dedi,aSozR1GKefY
6,çikolata mı,465.203488,470.323372,evet şimdi kenar için hazırlamış olduğum çikol...,J8K0vB6HgrY
7,çikolata mı,89.476286,92.242500,lokum mu çikolata mı deseler,aI0JSNFZ5bA
8,çikolata mı,379.474227,380.688454,üzerlerindeki çikolata mı aldım bakın çok güze...,LfHpjk6K2hw
9,çikolata mı,420.517727,421.315455,telefonda şeker mi alsam çikolata mı alsam bak...,0AOYUEuSiyw


In [ ]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,ama doktor,7037.567000,7039.472128,ama doktor dinlenmen gerektiğini söyledi bunda...,1yYN4FZDEGQ
1,ama doktor,4714.421000,4716.627500,ama doktor bahsetmişti zaten,N5rqYAWRdu4
2,ama doktor,2838.254000,2840.370667,ama doktor bey bize bol fındıkla kiraz suyu ön...,M6M7psZYUB4
3,ama doktor,468.777449,470.664388,ameliyatı kaç saat sürüyor diyorlar benimki be...,jYxyg1TrJZg
4,ama doktor,6140.483750,6142.482500,bir sevdiğin ama doktor vallaha aynen böyle dedi,z0cVY9navGM
5,ama doktor,6011.501000,6013.337862,ama doktor bir süre olur dedi,aSozR1GKefY
6,çikolata mı,464.603488,470.923372,evet şimdi kenar için hazırlamış olduğum çikol...,J8K0vB6HgrY
7,çikolata mı,88.876286,92.842500,lokum mu çikolata mı deseler,aI0JSNFZ5bA
8,çikolata mı,378.874227,381.288454,üzerlerindeki çikolata mı aldım bakın çok güze...,LfHpjk6K2hw
9,çikolata mı,419.917727,421.915455,telefonda şeker mi alsam çikolata mı alsam bak...,0AOYUEuSiyw


In [ ]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,ama doktor,7038,7039,ama doktor dinlenmen gerektiğini söyledi bunda...,1yYN4FZDEGQ
1,ama doktor,4714,4717,ama doktor bahsetmişti zaten,N5rqYAWRdu4
2,ama doktor,2838,2840,ama doktor bey bize bol fındıkla kiraz suyu ön...,M6M7psZYUB4
3,ama doktor,469,471,ameliyatı kaç saat sürüyor diyorlar benimki be...,jYxyg1TrJZg
4,ama doktor,6140,6142,bir sevdiğin ama doktor vallaha aynen böyle dedi,z0cVY9navGM
5,ama doktor,6012,6013,ama doktor bir süre olur dedi,aSozR1GKefY
6,çikolata mı,465,471,evet şimdi kenar için hazırlamış olduğum çikol...,J8K0vB6HgrY
7,çikolata mı,89,93,lokum mu çikolata mı deseler,aI0JSNFZ5bA
8,çikolata mı,379,381,üzerlerindeki çikolata mı aldım bakın çok güze...,LfHpjk6K2hw
9,çikolata mı,420,422,telefonda şeker mi alsam çikolata mı alsam bak...,0AOYUEuSiyw


In [ ]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ama doktor,7038,7039,ama doktor dinlenmen gerektiğini söyledi bunda...,1yYN4FZDEGQ,https://www.youtube.com/watch?v=1yYN4FZDEGQ&t=...
1,ama doktor,4714,4717,ama doktor bahsetmişti zaten,N5rqYAWRdu4,https://www.youtube.com/watch?v=N5rqYAWRdu4&t=...
2,ama doktor,2838,2840,ama doktor bey bize bol fındıkla kiraz suyu ön...,M6M7psZYUB4,https://www.youtube.com/watch?v=M6M7psZYUB4&t=...
3,ama doktor,469,471,ameliyatı kaç saat sürüyor diyorlar benimki be...,jYxyg1TrJZg,https://www.youtube.com/watch?v=jYxyg1TrJZg&t=...
4,ama doktor,6140,6142,bir sevdiğin ama doktor vallaha aynen böyle dedi,z0cVY9navGM,https://www.youtube.com/watch?v=z0cVY9navGM&t=...
5,ama doktor,6012,6013,ama doktor bir süre olur dedi,aSozR1GKefY,https://www.youtube.com/watch?v=aSozR1GKefY&t=...
6,çikolata mı,465,471,evet şimdi kenar için hazırlamış olduğum çikol...,J8K0vB6HgrY,https://www.youtube.com/watch?v=J8K0vB6HgrY&t=...
7,çikolata mı,89,93,lokum mu çikolata mı deseler,aI0JSNFZ5bA,https://www.youtube.com/watch?v=aI0JSNFZ5bA&t=89s
8,çikolata mı,379,381,üzerlerindeki çikolata mı aldım bakın çok güze...,LfHpjk6K2hw,https://www.youtube.com/watch?v=LfHpjk6K2hw&t=...
9,çikolata mı,420,422,telefonda şeker mi alsam çikolata mı alsam bak...,0AOYUEuSiyw,https://www.youtube.com/watch?v=0AOYUEuSiyw&t=...


In [ ]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{sheets.capitalize()}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [20]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_French_2 gram hybrid_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [21]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [22]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass